In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext

try:
    SparkContext.getOrCreate().stop()
except:
    pass

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Test") \
    .getOrCreate()

print(spark.version)



In [ ]:
# !curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/fhvhv_tripdata_2025-01.parquet

In [ ]:
df = spark.read.parquet("fhvhv_tripdata_2025-01.parquet")
df.show(5, False)

In [ ]:
df.schema

In [ ]:
partition_df = df.repartition(24)

In [ ]:
# partition_df.write.parquet('tripdata/2025/01')

In [ ]:
partition_df.printSchema()

In [ ]:
partition_df.select('PULocationID','DOLocationID','tips').filter(partition_df.tips > 0).show(5, False)

In [ ]:
from pyspark.sql import functions as F

def total_income(driver_pay,tips):
    if driver_pay is None:
        driver_pay = 0
    if tips is None:
        tips = 0
    return driver_pay + tips
total_income_udf = F.udf(total_income)
total_income_udf(partition_df.driver_pay, partition_df.tips) # 110

In [ ]:
partition_df \
    .withColumn('pickup_date', F.to_date(partition_df.pickup_datetime).cast("String")) \
    .withColumn('dropoff_date', F.to_date(partition_df.dropoff_datetime).cast("String")) \
    .withColumn('total_income', partition_df.driver_pay + partition_df.tips) \
    .select(
        'PULocationID',
        'DOLocationID',
        'pickup_date',
        'dropoff_date',
        'tips',
        'total_income'
    ) \
    .show(5, False)
